# S09_T02_Supervisat_Regressio

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import scipy.stats
from scipy.stats import norm
from scipy import stats
from scipy.stats import t
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from numpy.polynomial.polynomial import polyfit
from sklearn.decomposition import PCA
import matplotlib.cm as cm

## Exercici 1. Crea almenys tres models de regressió diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv

De l'exercici S09_T01, hem guardat com a csv el dataframe vols05, que està net de NaN i que conté els dummies de CancellationCode. L'importem i extraiem informació.

In [2]:
vols = pd.read_csv('//home/rusi/Escritorio/rubenIT/DataSources/vols05.csv')#importem i li assignem un nom de dataframe

In [3]:
print(vols.info())
print(vols.describe())
print(vols.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 35 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   Unnamed: 0.1       int64  
 2   Year               int64  
 3   Month              int64  
 4   DayofMonth         int64  
 5   DayOfWeek          int64  
 6   DepTime            float64
 7   CRSDepTime         int64  
 8   ArrTime            float64
 9   CRSArrTime         int64  
 10  UniqueCarrier      object 
 11  FlightNum          int64  
 12  TailNum            object 
 13  ActualElapsedTime  float64
 14  CRSElapsedTime     float64
 15  AirTime            float64
 16  ArrDelay           float64
 17  DepDelay           float64
 18  Origin             object 
 19  Dest               object 
 20  Distance           int64  
 21  TaxiIn             float64
 22  TaxiOut            float64
 23  Cancelled          int64  
 24  CancellationCode   object 
 25  Diverted          

In [4]:
vols.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Year', 'Month', 'DayofMonth',
       'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime',
       'UniqueCarrier', 'FlightNum', 'TailNum', 'ActualElapsedTime',
       'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest',
       'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode',
       'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
       'LateAircraftDelay', 'A', 'B', 'C', 'N'],
      dtype='object')

In [5]:
vols.describe(include = "all")

,Unnamed: 0,Unnamed: 0.1,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,...,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,A,B,C,N
count,1.936758e+06,1.936758e+06,1936758.0,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,...,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.683785e+05,3.341651e+06,2008.0,6.111106e+00,1.575347e+01,3.984827e+00,1.518534e+03,1.467473e+03,1.604230e+03,1.634225e+03,...,4.003598e-03,1.235367e+01,2.385512e+00,9.675607e+00,5.805836e-02,1.629374e+01,1.270164e-04,1.585123e-04,4.130614e-05,9.996732e-01
std,5.590940e+05,2.066065e+06,0.0,3.482546e+00,8.776272e+00,1.995966e+00,4.504853e+02,4.247668e+02,5.557685e+02,4.646347e+02,...,6.314722e-02,3.613493e+01,1.734036e+01,2.808958e+01,1.623934e+00,3.585904e+01,1.126944e-02,1.258917e-02,6.426854e-03,1.807562e-02
min,0.000000e+00,0.000000e+00,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.841892e+05,1.517452e+06,2008.0,3.000000e+00,8.000000e+00,2.000000e+00,1.203000e+03,1.135000e+03,1.313000e+03,1.325000e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,9.683785e+05,3.242558e+06,2008.0,6.000000e+00,1.600000e+01,4.000000e+00,1.545000e+03,1.510000e+03,1.714000e+03,1.705000e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.452568e+06,4.972467e+06,2008.0,9.000000e+00,2.300000e+01,6.000000e+00,1.900000e+03,1.815000e+03,2.030000e+03,2.014000e+03,...,0.000000e+00,1.000000e+01,0.000000e+00,6.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00


Prescindim de les columnes Unnamed, que són repeticions de l'índex.

In [6]:
vols02=vols.drop(columns=["Unnamed: 0","Unnamed: 0.1"])

In [7]:
vols02.describe(include="all")

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,A,B,C,N
count,1936758.0,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1936758,1.936758e+06,...,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06,1.936758e+06
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,377602,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2008.0,6.111106e+00,1.575347e+01,3.984827e+00,1.518534e+03,1.467473e+03,1.604230e+03,1.634225e+03,NaN,2.184263e+03,...,4.003598e-03,1.235367e+01,2.385512e+00,9.675607e+00,5.805836e-02,1.629374e+01,1.270164e-04,1.585123e-04,4.130614e-05,9.996732e-01
std,0.0,3.482546e+00,8.776272e+00,1.995966e+00,4.504853e+02,4.247668e+02,5.557685e+02,4.646347e+02,NaN,1.944702e+03,...,6.314722e-02,3.613493e+01,1.734036e+01,2.808958e+01,1.623934e+00,3.585904e+01,1.126944e-02,1.258917e-02,6.426854e-03,1.807562e-02
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2008.0,3.000000e+00,8.000000e+00,2.000000e+00,1.203000e+03,1.135000e+03,1.313000e+03,1.325000e+03,NaN,6.100000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,2008.0,6.000000e+00,1.600000e+01,4.000000e+00,1.545000e+03,1.510000e+03,1.714000e+03,1.705000e+03,NaN,1.543000e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,2008.0,9.000000e+00,2.300000e+01,6.000000e+00,1.900000e+03,1.815000e+03,2.030000e+03,2.014000e+03,NaN,3.422000e+03,...,0.000000e+00,1.000000e+01,0.000000e+00,6.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00


Preparem *x* i *y* train, amb el target "ArrDelay". Ho farem amb 2/3 de les mostres per entrenar, i 1/3 per test. Eliminem de *x* els atributs categòrics.

In [23]:
x=vols02.drop(columns=["ArrDelay","TailNum","Origin","Dest","CancellationCode","UniqueCarrier"])
y=vols02.ArrDelay
print(x.describe(include="all"))
print(y.describe(include="all"))

            Year         Month    DayofMonth     DayOfWeek       DepTime  \
count  1936758.0  1.936758e+06  1.936758e+06  1.936758e+06  1.936758e+06   
mean      2008.0  6.111106e+00  1.575347e+01  3.984827e+00  1.518534e+03   
std          0.0  3.482546e+00  8.776272e+00  1.995966e+00  4.504853e+02   
min       2008.0  1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
25%       2008.0  3.000000e+00  8.000000e+00  2.000000e+00  1.203000e+03   
50%       2008.0  6.000000e+00  1.600000e+01  4.000000e+00  1.545000e+03   
75%       2008.0  9.000000e+00  2.300000e+01  6.000000e+00  1.900000e+03   
max       2008.0  1.200000e+01  3.100000e+01  7.000000e+00  2.400000e+03   

         CRSDepTime       ArrTime    CRSArrTime     FlightNum  \
count  1.936758e+06  1.936758e+06  1.936758e+06  1.936758e+06   
mean   1.467473e+03  1.604230e+03  1.634225e+03  2.184263e+03   
std    4.247668e+02  5.557685e+02  4.646347e+02  1.944702e+03   
min    0.000000e+00  0.000000e+00  0.000000e+00  1.0000

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=4)

In [25]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1297627, 27) (639131, 27) (1297627,) (639131,)


In [26]:
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1297627 entries, 84384 to 1000570
Data columns (total 27 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Year               1297627 non-null  int64  
 1   Month              1297627 non-null  int64  
 2   DayofMonth         1297627 non-null  int64  
 3   DayOfWeek          1297627 non-null  int64  
 4   DepTime            1297627 non-null  float64
 5   CRSDepTime         1297627 non-null  int64  
 6   ArrTime            1297627 non-null  float64
 7   CRSArrTime         1297627 non-null  int64  
 8   FlightNum          1297627 non-null  int64  
 9   ActualElapsedTime  1297627 non-null  float64
 10  CRSElapsedTime     1297627 non-null  float64
 11  AirTime            1297627 non-null  float64
 12  DepDelay           1297627 non-null  float64
 13  Distance           1297627 non-null  int64  
 14  TaxiIn             1297627 non-null  float64
 15  TaxiOut            1297627 n

## 1r mètode: Linear Regression

In [41]:
lm = linear_model.LinearRegression()
model = lm.fit(x,y)

In [46]:
predictions = lm.predict(x)
print("Predictions: " + str(predictions))
print("Score or R^2 de train: " + str(lm.score(x,y)))
print("Coefficient: " + str(lm.coef_))
print("Intercept: " + str(lm.intercept_))

Predictions: [-2.81271377  1.98035663  5.74663751 ... 98.54910934  3.82583212
  0.40925135]
Score or R^2 de train: 0.9947825425307268
Coefficient: [-4.52775230e-14 -8.27401904e-03  1.12753522e-03 -3.09888710e-03
 -1.89733176e-04  4.00379087e-05 -4.45443075e-05  2.96271730e-05
 -2.96509240e-05  1.34146007e-01 -2.84455923e-01  1.17761675e-01
  2.85365272e-01  3.19204818e-03  1.59968494e-01  1.52633566e-01
  2.21135916e-02  1.81994144e+01  7.05057714e-01  7.07102473e-01
  7.05888811e-01  6.91223937e-01  7.02423467e-01  3.13258569e+00
  3.97447238e+00 -7.08494448e+00 -2.21135916e-02]
Intercept: 2.6788699047290265


**Conclusió**: *score* és el percentatge de variança explicada de la predicció. Observem que és molt elevada, del 99,47%. Això ens fa pensar que estem en un cas de overfitting (a partir del 90% s'assumeix que és així).